
# Data Input

The datasets used in this project are available from ['The Cancer Genome Atlas' (TCGA)](www.http://cancergenome.nih.gov/) consortium.  Clinical and RNA-seq gene count data sets can be downloaded via a provided data portal, or using an R package called [TCGA2STAT](https://cran.r-project.org/web/packages/TCGA2STAT/index.html).  The R script used to download the data sets for this project and write files locally in [feather](https://github.com/wesm/feather)  format are available in the project repository [here](https://github.com/CCThompson82/MLE_capstone/tree/master/Dataset_setup).  Datasets are also stored in this project's repository [here](https://github.com/CCThompson82/MLE_capstone/tree/master/feather_files).  The R script is not necessary for dataset download, but can be run locally to obtain the most up to date clinical information from TCGA.  

In [1]:
%run -i Dataset_cleanup/arrange_DF.py

Clinical data set imported!
Features: 21 
Patients: 499

 The following features do not provide any information: 
 ['Composite.Element.REF' 'ethnicity' 'gender' 'pathologicstage'
 'pathologyMstage' 'tumortissuesite'] 

Variables that are not known at initial diagnosis: 
 ['daystodeath' 'daystolastfollowup' 'daystopsa' 'histologicaltype'
 'numberoflymphnodes' 'pathologyTstage' 'radiationtherapy' 'residualtumor'
 'vitalstatus'] 

Variables that are known at the time of diagnosis:
 ['clinical_index' 'dateofinitialpathologicdiagnosis' 'gleasonscore'
 'pathologyNstage' 'psavalue' 'race' 'yearstobirth']


Gene Counts data set imported!
Features: 20501 
Patients: 497


Transforming gene counts to transcript per million (TPM)

Transformation Successful!

497 Gene count estimate profiles have been transformed from gene counts to transcripts per million reads (TPM)


# Data Exploration
## Clinical Information and metastasis label
There is missing clinical data in many of the features, including what will become the outcome label ('pathologyNstage' - metastasis state).  The series **'y_all'** is the full list of pathologyNstage, where 'n1' represents metastasis, and 'n0' represents no metastasis observed to date.  Some observations have no metastasis state recorded and are represented by NaN in y_all.  These are removed for the trimmed **'y'** series.  

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
D = (('n0', y_all[y_all =='n0'].shape[0]), 
     ('n1', y_all[y_all =='n1'].shape[0]), 
     ('missing' , y_all[y_all.isnull()].shape[0]))
exp_fig = plt.figure(figsize=(5 , 5))
A = exp_fig.add_subplot(1,1,1)
ind = A.bar(range(0,3), height= [D[0][1], D[1][1], D[2][1]], align='center', color = 'grey')
A.set_xticks(range(0,3))
A.set_ylabel('Frequency', fontsize=16)
A.set_xlabel('Metastasis state', fontsize=16)
A.set_ylim(0,400)
A.set_xticklabels([D[0][0], D[1][0], D[2][0]])  #Must be a better way to do this as tuples (D, in this case) are not ordered?
ep = plt.show
exp_fig.savefig('Figures/LabelCount.png')

#### Gleason Score versus Metastasis analysis
The Gleason score is the gold-standard diagnostic test for cancer severity, but is not highly correlative with metastatic disease.  

In [ ]:
clinical['gleasonscore'] = pd.to_numeric(clinical['gleasonscore'], errors= 'coerce')
clinical['gleasonscore'].isnull().any()

In [ ]:
GS_hist = plt.figure(figsize=(5,5))
A = GS_hist.add_subplot(1,1,1)
sub = (('n1', ))

subs = [clinical.loc[y_all[y_all == 'n0'].index]['gleasonscore'],
        clinical.loc[y_all[y_all == 'n1'].index]['gleasonscore'],
        clinical.loc[y_all[y_all.isnull()].index]['gleasonscore']]

bins = [5.99, 6.99, 7.99, 8.99, 9.99, 10.99]
A.hist(subs, bins, label =['n0', 'n1', 'unknown'], color = ['blue','red','grey'], stacked = True)
A.set_ylim(0,275)
A.set_ylabel('Count', fontsize=16)
A.set_xlabel('Gleason Score', fontsize =16)
A.legend()
plt.show
GS_hist.savefig('Figures/GleasonHist.png')

It appears that no metastases are recorded in those specimens graded at a Gleason score of 6.  This represents an opportunity to replace the missing data label with the most likely pathology state, n0, to more efficiently use the small data set.  

In [ ]:
"""Define the indices where gleasonscore == 6 and pathologyNstage is null"""
set(y_all[y_all.isnull()].index).intersection(list(clinical[clinical['gleasonscore'] == 6].index))

In [ ]:
y_all.loc[set(y_all[y_all.isnull()].index).intersection(list(clinical[clinical['gleasonscore'] == 6].index))] = 'n0'

In [ ]:
GS_hist = plt.figure(figsize=(5,5))
A = GS_hist.add_subplot(1,1,1)
sub = (('n1', ))

subs = [clinical.loc[y_all[y_all == 'n0'].index]['gleasonscore'],
        clinical.loc[y_all[y_all == 'n1'].index]['gleasonscore'],
        clinical.loc[y_all[y_all.isnull()].index]['gleasonscore']]

bins = [5.99, 6.99, 7.99, 8.99, 9.99, 10.99]
A.hist(subs, bins, label =['n0', 'n1', 'unknown'], color = ['blue','red','grey'], stacked = True)
A.set_ylim(0,275)
A.set_ylabel('Count', fontsize=16)
A.set_xlabel('Gleason Score', fontsize =16)
A.legend()
plt.show
GS_hist.savefig('Figures/GleasonHist2.png')

## Gene Activation (Gene counts) Dataset
The starting dataset, **'X_all'**, includes the transformed transcript per million (TPM) estimates for all RNA-seq profiles.  However some of the observations in this set do not have corresponding y_labels, as the clinical data set contains missing information.

Therefore X_all was trimmed to include only those observations where a finite y label exists, to yield **'X'**.  

In [ ]:
print("Total observations in original dataset:",clinical.shape[0])

not_labeled = y_all[y_all.isnull()] 
y = y_all[y_all.notnull()]

print("\nObservations with metastasis label:",y.shape[0])
print("Unlabeled observations (removed:)",not_labeled.shape[0])


In [ ]:
"""Limit X to only observations where a target label is present."""
X = X_all.loc[set(y.index).intersection(X_all.index)]  #Only observations that also have a known metastasis state are kept.
y = y.loc[set(X.index).intersection(y.index)]
print("X dimensions:",X.shape,"\ny dimensions:",y.shape)

In [ ]:
X_no_y = X_all.loc[list(not_labeled.index)]
print("Dimensions of unlabeled dataset:",X_no_y.shape)

In [ ]:
"""Check to determine if any clinical information is missing"""
X.isnull().values.any()

## Data Visualization

In [ ]:
gleason = clinical['gleasonscore']
#print(gleason.shape)
age = pd.to_numeric(clinical['yearstobirth'], errors = 'coerce')
age.fillna(value = np.mean(age), inplace=True)
psa = pd.to_numeric(clinical['psavalue'], errors= 'coerce')
psa.fillna(value = np.mean(psa), inplace =True)
#gleason = gleason.loc[y.index]

clinicalDF_all = pd.DataFrame({'gleason': gleason,
                            'age':age ,
                            'psa' : psa,
                            'y' : y_all}, index=X_all.index)
clinicalDF = clinicalDF_all.loc[y.index, :]

In [ ]:
clinicalDF.drop(['y'], axis=1, inplace = True)

In [ ]:
clinicalDF = clinicalDF.reindex(y.index)

In [ ]:
sm = pd.scatter_matrix(clinicalDF, 
                       alpha=0.25, 
                       figsize= (10,10), 
                       diagonal = 'kde', 
                       c = y.replace({'n1': 'red','n0':'blue'}), 
                       s = 125)

plt.savefig("Figures/ClinScatterMatrix.png")

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
K_selector = SelectKBest(f_classif, k = 'all') # using k=all here and will filter based on F-stat later.  
K = K_selector.fit(X, y)
K_df = pd.DataFrame({'Gene':X.columns, 'F_score':K.scores_})
K_df.dropna(inplace=True)

In [ ]:
F_fig = plt.figure(figsize=(15,5))
A = F_fig.add_subplot(1,2,1)
A.hist(K_df['F_score'], bins=range(0,45,1), facecolor='green')
#A.set_yscale('log')
A.set_ylabel('Count', fontsize=16)
A.set_xlabel("F-statistic", fontsize=16)
F_fig.suptitle("Distribution of F-statistics for metatastasis state group comparison of gene expression", fontsize = 18)
B = F_fig.add_subplot(1,2,2)
B.hist(K_df['F_score'], bins=range(0,45,1), facecolor='green')
B.set_yscale('log')
B.set_ylabel('Count', fontsize=16)
B.set_xlabel("F-statistic", fontsize=16)
#B.set_title("Distribution of F-statistics for metatastasis state group comparison of gene expression", fontsize = 18)
plt.show()
F_fig.savefig("Figures/FDist.png")

# Analysis
## Prepare for Analysis
### Generate Scoring Functions

In [ ]:
from sklearn.metrics import matthews_corrcoef, fbeta_score, classification_report, log_loss
from sklearn.model_selection import cross_val_score

In [ ]:
"""Scorers needed throughout code:"""
from sklearn.metrics import make_scorer
LL_scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True, needs_threshold=False)
MCC_scorer = make_scorer(matthews_corrcoef, greater_is_better=True, needs_proba=False, needs_threshold=False)
fbeta_scorer = make_scorer(fbeta_score, greater_is_better=True, needs_proba=False, needs_threshold=False, pos_label='n1', beta = 2)

### Import Logistic Regression Algorithms

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

### Set Seed and Folds for Cross Validation

In [ ]:
seed = 123
folds = 4

### Handles for Graph legends

In [ ]:
import matplotlib.patches as mpatches
blue = mpatches.Patch(color ='blue', label ='n0')
red = mpatches.Patch(color='red', label = 'n1')

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
train_k = round(len(y)*0.70)
test_k = len(y) - train_k

In [ ]:
"""Split samples into train and test sets, stratified by Gleason score"""
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size= test_k, 
                                                    train_size = train_k,
                                                    random_state = seed,
                                                    stratify = clinicalDF['gleason'])

In [ ]:
clinicalDF_train = clinicalDF.loc[X_train.index,:]
clinicalDF_test = clinicalDF.loc[X_test.index,:]

## Benchmark

### Clinical Information Benchmark

In [ ]:
clf_LR_clin = LogisticRegression(penalty='l2',
                              dual=False,
                              tol=0.0001,
                              C=1,
                              fit_intercept=True,
                              intercept_scaling=1,
                              class_weight='balanced',
                              random_state=123,
                              solver='liblinear',
                              max_iter=100,
                              multi_class='ovr',
                              verbose=0,
                              warm_start=False,
                              n_jobs=1)

clf_LR_clin.fit(clinicalDF_train, y_train)  #use training set for model learning

In [ ]:
clf_LR_clin = LogisticRegression(penalty='l2',
                              dual=False,
                              tol=0.0001,
                              C=1,
                              fit_intercept=True,
                              intercept_scaling=1,
                              class_weight='balanced',
                              random_state=123,
                              solver='liblinear',
                              max_iter=100,
                              multi_class='ovr',
                              verbose=0,
                              warm_start=False,
                              n_jobs=1)

clf_LR_clin.fit(clinicalDF_train, y_train)  #use training set for model learning

bench_fig = plt.figure(figsize=(15,5))
A= bench_fig.add_subplot(1,2,1)
B = bench_fig.add_subplot(1,2,2)
A.scatter(clinicalDF_train.loc[:,'gleason'], 
          clf_LR_clin.predict_proba(clinicalDF_train)[:,1], 
          color = y_train.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.3,
          s = 25)
A.set_ylabel('Benchmark model probability of metastasis')
A.set_ylim(0,1)
B.set_ylim(0,1)
A.set_xlabel('Gleason')
plt.suptitle('Prognosis of metastasis risk based on features available at presentation', fontsize=20)
A.set_title('Training Set')
B.set_title('Validation Set')
B.scatter(clinicalDF_test.loc[:,'gleason'],
          clf_LR_clin.predict_proba(clinicalDF_test)[:,1],
          color = y_test.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.5,
          s = 25)
B.set_xlabel('Gleason')
B.set_ylabel('Benchmark model probability of metastasis')
bench_fig.legend(labels = ['n0', 'n1'], handles = [blue, red], loc = 5)
bench_fig.suptitle("Benchmark Model", fontsize=20)
plt.show
bench_fig.savefig('Figures/benchmark.png')

In [ ]:
print(pd.DataFrame({'features':clinicalDF.columns, 'coefficient': clf_LR_clin.coef_[0]}))

In [ ]:
"Benchmark Performance on Train dataset"
print(clf_LR_clin)
print('\nNull F beta: ', fbeta_score(y_train, clf_LR_clin.predict(clinicalDF_train), pos_label='n1',beta=2))
print('\nMCC: ',matthews_corrcoef(y_train, clf_LR_clin.predict(clinicalDF_train)),"\n")
print(classification_report(y_train, clf_LR_clin.predict(clinicalDF_train), labels = ['n0','n1']))
print('\nLogLoss: ', log_loss(y_train.replace({'n0':0, 'n1':1}), 
         clf_LR_clin.predict_proba(clinicalDF_train)[:,1]))

training_stat = pd.DataFrame({'F2': fbeta_score(y_train, clf_LR_clin.predict(clinicalDF_train), pos_label='n1',beta=2),
                            'MCC': matthews_corrcoef(y_train, clf_LR_clin.predict(clinicalDF_train)),
                            'LogLoss':log_loss(y_train.replace({'n0':0, 'n1':1}), 
                                      clf_LR_clin.predict_proba(clinicalDF_train)[:,1])}, index = ['clinical_only'])

In [ ]:
"Benchmark Performance on Test dataset"
print(clf_LR_clin)
print('\nNull F beta: ', fbeta_score(y_test, clf_LR_clin.predict(clinicalDF_test), pos_label='n1',beta=2))
print('\nMCC: ',matthews_corrcoef(y_test, clf_LR_clin.predict(clinicalDF_test)),"\n")
print(classification_report(y_test, clf_LR_clin.predict(clinicalDF_test), labels = ['n0','n1']))
print('\nLogLoss: ', log_loss(y_test.replace({'n0':0, 'n1':1}), 
         clf_LR_clin.predict_proba(clinicalDF_test)[:,1]))

metric_stat = pd.DataFrame({'F2': fbeta_score(y_test, clf_LR_clin.predict(clinicalDF_test), pos_label='n1',beta=2),
                            'MCC': matthews_corrcoef(y_test, clf_LR_clin.predict(clinicalDF_test)),
                            'LogLoss':log_loss(y_test.replace({'n0':0, 'n1':1}), 
                                      clf_LR_clin.predict_proba(clinicalDF_test)[:,1])}, index = ['clinical_only'])

## Feature Reduction
### Import Functions

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Gini Importance Filter

In [ ]:
clf_RF = RandomForestClassifier(n_estimators=1000, 
                                criterion='gini', 
                                max_depth=3, 
                                min_samples_split=30, 
                                min_samples_leaf=5, 
                                min_weight_fraction_leaf=0.0, 
                                max_features= 'auto', 
                                max_leaf_nodes=None, 
                                bootstrap=True, 
                                oob_score=False, 
                                n_jobs=1, 
                                random_state=seed, 
                                verbose=0, 
                                warm_start=False, 
                                class_weight='balanced')


In [ ]:
clf_RF.fit(X, y)
Gini_DF = pd.DataFrame({'Gini' :clf_RF.feature_importances_}, index=X.columns).sort_values(by = ['Gini'], axis = 0, ascending = False)
#Gini_DF.reset_index(inplace=True)
print(Gini_DF.head())

In [ ]:
"""Set k number of genes to retain"""
k = 100

### Stability Experiment

In [ ]:
genes = Gini_DF.index[0:k] # gene list from first instance  
for x in [5, 54, 543, 5432, 54321] :
    clf_RF = RandomForestClassifier(n_estimators=1000, 
                                criterion='gini', 
                                max_depth=3, 
                                min_samples_split=30, 
                                min_samples_leaf=5, 
                                min_weight_fraction_leaf=0.0, 
                                max_features= 'auto', 
                                max_leaf_nodes=None, 
                                bootstrap=True, 
                                oob_score=False, 
                                n_jobs=1, 
                                random_state=x, 
                                verbose=0, 
                                warm_start=False, 
                                class_weight='balanced')
    clf_RF.fit(X, y)
    Gini_DF = pd.DataFrame({'Gini' :clf_RF.feature_importances_}, index=X.columns).sort_values(by = ['Gini'], axis = 0, ascending = False)
    genes = set(genes).intersection(Gini_DF.index[0:k])
    print('Number of Consistent (Stable) Genes:',len(genes))

In [ ]:
"""Subset X to include only the k best features from the Gini Importance table"""
X_k = X.loc[:,genes]

In [ ]:

fig = plt.figure(figsize=(12,8))
for i in range(0,len(X_k.columns),1) :
    ax = fig.add_subplot(np.ceil(len(X_k.columns)/5),5,i+1)
    df0 = list(X_k.ix[y.loc[y=='n0'].index,i])
    df1 = list(X_k.ix[y.loc[y=='n1'].index,i])
    data = [df0,df1]
    A = ax.boxplot(data, positions = [1,2], vert = False, widths = 0.5, patch_artist=True)
    plt.setp(A['boxes'][1], color='red')
    plt.setp(A['boxes'][0], color='blue')
    plt.setp(A['medians'], color = 'black')
    plt.setp(A['whiskers'], color = 'black')
    plt.setp(A['fliers'], color = 'black', marker = '.', markersize = 3)
    ax.yaxis.set_major_formatter(plt.NullFormatter())
    ax.set_xlabel(X_k.columns[i], fontsize=12)
    ax.set_xscale('log')
fig.tight_layout()
fig.legend(labels = ['n0', 'n1'], handles= [blue,red], ncol = 2, loc = 'lower center', bbox_to_anchor = (.945, 1.05))
fig.text(-.005, 0.5, 'Metastasis state', ha='center', va='center', fontsize=24,rotation='vertical')
fig.text(.5, -.025, 'Transcripts Per Million (TPM)', ha='center', va='center', fontsize=24)
fig.savefig('Figures/boxplots.png', dpi = 300, bbox_inches='tight')
plt.show()


### Transform reduced feature dataframe to standard scale

In [ ]:
scaler = StandardScaler()
Xs = pd.DataFrame(scaler.fit_transform(X_k), columns = X_k.columns, index = X_k.index)


### PCA transformation
#### Train on the scaled feature set

In [ ]:
components = 3

In [ ]:
pca = PCA(n_components = components, whiten=False)
pca.fit(Xs)

#### Transform with the trained PCA function

In [ ]:
Xpc = pd.DataFrame(pca.transform(Xs), columns = range(0,components,1), index = X.index)

#### Visualize the PCA transformation

In [ ]:
%run -i 'support files/renders_PCA.py'
pca_results(Xs, pca)
plt.savefig('Figures/pcaEV.png', dpi =500)

In [ ]:

sm = pd.scatter_matrix(Xpc, 
                       alpha=0.5, 
                       figsize= (10,10), 
                       diagonal = 'kde', 
                       c = y.replace({'n1': 'red','n0':'blue'}), 
                       s = 50)
plt.savefig("Figures/pcaScatter.png", dpi = 300)

#### Appropriate the PCA transformed dataset via the current train, test split index

In [ ]:
X_train, X_test = Xpc.loc[X_train.index, :], Xpc.loc[X_test.index, :]

## Train the Logistic Regression Algorithm
Training with the first three principle components with cross validation to determine the optimal C parameter.

In [ ]:
clf_LR = LogisticRegression(penalty='l2',
                              dual=False,
                              tol=0.0001,
                              C=1,
                              fit_intercept=True,
                              intercept_scaling=1,
                              class_weight='balanced',
                              random_state=123,
                              solver='liblinear',
                              max_iter=100,
                              multi_class='ovr',
                              verbose=0,
                              warm_start=False,
                              n_jobs=1)
clf_LR.fit(X_train, y_train)

### Visualize Model Performance on Training and Test sets

In [ ]:
%run -i 'Support Files/capstone_graphs.py'
fig.savefig('Figures/PC3.png')

### Measure Performance on the held-out test set

In [ ]:
run_name = '3_PCs'
%run -i 'Support Files/capstone_performance.py'

## Optimize the Logistic Regression Model

In [ ]:
clf_LR = LogisticRegressionCV(Cs=10, 
                              fit_intercept=True, 
                              cv=folds, 
                              dual=False, 
                              penalty='l2', 
                              scoring='log_loss', 
                              solver='liblinear', 
                              tol=0.0001, 
                              max_iter=100, 
                              class_weight='balanced', 
                              n_jobs=1, 
                              verbose=0, 
                              refit=True, 
                              intercept_scaling=1.0, 
                              multi_class='ovr', 
                              random_state=seed)
clf_LR.fit(X_train, y_train)

In [ ]:
print("The optimal C term: ",clf_LR.C_)

### Visualize Model Performance on Training and Test sets

In [ ]:
%run -i 'Support Files/capstone_graphs.py'
fig.savefig('Figures/optPC3.png')

### Measure Performance on the held-out test set

In [ ]:
run_name = '3_PCs_optimized'
%run -i 'Support Files/capstone_performance.py'

### Incorporate Gleason score

In [ ]:
X_train.loc[:,'gleason'] = clinicalDF.loc[X_train.index,'gleason']
X_test.loc[:,'gleason'] = clinicalDF.loc[X_test.index, 'gleason']

### Retrain Logistic Regression model with CV for C term

In [ ]:
clf_LR = LogisticRegressionCV(Cs=10, 
                              fit_intercept=True, 
                              cv=folds, 
                              dual=False, 
                              penalty='l2', 
                              scoring='log_loss', 
                              solver='liblinear', 
                              tol=0.0001, 
                              max_iter=100, 
                              class_weight='balanced', 
                              n_jobs=1, 
                              verbose=0, 
                              refit=True, 
                              intercept_scaling=1.0, 
                              multi_class='ovr', 
                              random_state=seed)
clf_LR.fit(X_train, y_train)

#### Visualize Model Performance
Trained on First 3 PCs + gleason score

In [ ]:
%run -i 'Support Files/capstone_graphs.py'
plt.savefig('Figures/PC3Gleason.png')

In [ ]:
sm = pd.scatter_matrix(X_train, 
                       alpha=0.5, 
                       figsize= (12,12), 
                       diagonal = 'kde', 
                       c = y_train.replace({'n1': 'red','n0':'blue'}), 
                       s = 50)
plt.savefig('Figures/FeatureSM.png')

#### Performance Measure

In [ ]:
run_name = 'Final_Model'
%run -i 'Support Files/capstone_performance.py'

#### Is each feature important for model performance?

In [ ]:
pd.DataFrame({'Component': X_train.columns, 'coef': clf_LR.coef_[0]})

In [ ]:
%run -i 'Support Files/final_graph.py'
fig.savefig('Figures/FF.png')

## Conclusions and Discussion

### Function for Prostate Cancer Metastasis Risk

In [ ]:
def risk_function(barcode, DF, clinDF, genes, scaler, fit_pca, fit_clf) :
    self = DF.loc[barcode,genes].reshape(1,-1)
    self = pd.DataFrame(scaler.transform(self))
    self = pd.DataFrame(fit_pca.transform(self), index = [barcode])
    self['gleason'] = clinDF.loc[barcode,'gleasonscore']
    return(fit_clf.predict_proba(self)[0,1])
    



In [ ]:
risk = []
for x in not_labeled.index :
    risk.append(risk_function(barcode = x,
              DF = X_all,
              clinDF = clinical,
              genes = genes,
              fit_pca = pca,
              scaler = scaler,
              fit_clf = clf_LR))
model_risk = pd.Series(risk, index=not_labeled.index)

In [ ]:
clinicalDF_all.drop(['y'], axis=1, inplace = True)

In [ ]:
bench_risk = pd.Series(clf_LR_clin.predict_proba(clinicalDF_all.loc[not_labeled.index,:])[:,1])

In [ ]:
n0_question = X_all.loc[y[y!='n1'].index, :]

In [ ]:
risk = []
for x in n0_question.index :
    risk.append(risk_function(barcode = x,
              DF = n0_question,
              clinDF = clinical,
              genes = genes,
              fit_pca = pca,
              scaler = scaler,
              fit_clf = clf_LR))
n0_risk = pd.Series(risk, index=n0_question.index)

### Visualize unlabeled and 'non-metastatic' cases from the TCGA cohort

In [ ]:
%run -i 'Support Files/free-form.py'
fig.savefig('Figures/missing.png')

In [ ]:
%run -i 'Support Files/free-form2.py'
fig.savefig('Figures/N0Analysis.png')

It is important to remember that the dataset utilized in this project is longitudinal and active, and indeed some cases listed as 'n0' will eventually be diagnosed as 'n1'.  Even though the model was trained on such false-negative labeled cases, some of the 'n0' class are still predicted as highly likely to metastasize.  If the dataset were linked somehow to patient records (for ethical purposes, no such link exists), then these patients should be monitored with extreme care for signs of metastasis based on the predictions made by the project model.

In [ ]:
n0_risk[n0_risk > 0.75].sort_values(ascending = False)

### Sensitivity Analysis

In [ ]:
benign = feather.read_dataframe('feather_files/benign.feather')

In [ ]:
benign.set_index(['benign_index'], inplace = True)

In [ ]:
benign.shape

In [ ]:
benign_tpm = transformation(benign)

In [ ]:
risk = []
for x in benign.index :
    risk.append(risk_function(barcode = x,
              DF = benign,
              clinDF = clinical,
              genes = genes,
              fit_pca = pca,
              scaler = scaler,
              fit_clf = clf_LR))
benign_risk = pd.Series(risk, index=benign.index)

In [ ]:

fig = plt.figure(figsize=(8,4))
A = fig.add_subplot(1,1,1)
A.hist(benign_risk.sort_values(ascending=False),
       bins=12,
       facecolor='blue')
#A.set_yscale('log')
A.set_ylabel('Count', fontsize=16)
A.set_xlim(0,1)
A.set_xlabel("Model Probability of Metastasis", fontsize=16)
fig.suptitle("Distribution of metastasis probability for benign samples", fontsize = 14)
plt.show()
fig.savefig("Figures/Sensitivity.png")